# Задание

1. Используя Gym Atari, выберите любую игру с более чем 4-мя действиями.
2. Обучите игру на стратегии градиента стратегии (градиентной политики).
3. Создайте видео из нескольких игровых эпизодов, в котором модель достигла максимального вознаграждения.
  

In [1]:
#@title Установка зависимостей
!pip install "gymnasium[atari]"
!pip install autorom[accept-rom-license]
!pip install moviepy
!pip install numpy
!pip install opencv-python
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=autorom_accept_rom_license-0.6.1-py3-none-any.whl size=446709 sha256=5a063331ddfb786147a2c055c36bedecc8948076fd15bf7943ca9b2926fe7f85
  Stored in directory: /root/.cache/pip/wheels/99/f1/ff/c6966c034a8259164bdc9deb4d1ea839f119474638100e6645
Successfully built AutoROM.accept-rom-license


In [2]:
#@title Импорты
import gymnasium as gym
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from moviepy.editor import ImageSequenceClip
import base64
from IPython.display import HTML, display
import ale_py

gym.register_envs(ale_py)

torch.manual_seed(42)
np.random.seed(42)

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [3]:
#@title Константы и параметры

GAME_NAME = "ALE/MsPacman-v5"
FULL_ACTION_SPACE = False

MAX_STEPS = 2000          # Длительность эпизода
STATE_SIZE = 32           # Размер препроцессированного изображения 32x32
FLAT_SIZE = STATE_SIZE * STATE_SIZE   # 1024

# Параметры Policy Gradient
GAMMA = 0.99              # Фактор дисконтирования
LEARNING_RATE = 0.001     # Скорость обучения
N_EPISODES = 500         # Количество эпизодов для обучения

# Определение числа действий
env_tmp = gym.make(GAME_NAME, full_action_space=FULL_ACTION_SPACE)
N_ACTIONS = env_tmp.action_space.n
env_tmp.close()

print("Игра:", GAME_NAME)
print("Действий:", N_ACTIONS)
print("Размер входа (32x32):", FLAT_SIZE)

Игра: ALE/MsPacman-v5
Действий: 9
Размер входа (32x32): 1024


In [4]:
#@title Препроцессинг и функция видео

def preprocess(state, size=STATE_SIZE):
    """Grayscale → Resize → Normalize → Tensor."""
    gray = cv2.cvtColor(state, cv2.COLOR_RGB2GRAY)
    small = cv2.resize(gray, (size, size), interpolation=cv2.INTER_AREA)
    # Преобразование в PyTorch Tensor
    tensor = torch.from_numpy(small.flatten()).float() / 255.0
    return tensor

def show_video(frames, filename="episode.mp4", fps=30):
    clip = ImageSequenceClip(frames, fps=fps)
    clip.write_videofile(filename, codec="libx264", audio_codec="aac", verbose=False)

    video_bytes = open(filename, "rb").read()
    encoded = base64.b64encode(video_bytes).decode()

    display(HTML(f"""
    <video width=600 controls>
        <source src="data:video/mp4;base64,{encoded}" type="video/mp4">
    </video>
    """))

In [5]:
#@title Модель политики (Нейронная сеть)

class PolicyNetwork(nn.Module):
    """Простая нейронная сеть для отображения состояния в распределение действий."""
    def __init__(self, input_size, output_size):
        super(PolicyNetwork, self).__init__()
        # Используем линейный слой для простоты с входным размером 1024 (32*32)
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # Выход - это логиты для распределения действий
        return self.fc2(x)

policy = PolicyNetwork(FLAT_SIZE, N_ACTIONS)
optimizer = optim.Adam(policy.parameters(), lr=LEARNING_RATE)

In [7]:
#@title Policy Gradient

def run_episode_pg(env, policy, max_steps=MAX_STEPS, record=False):
    state, _ = env.reset()
    log_probs = []
    rewards = []
    frames = []
    total_reward = 0

    for t in range(max_steps):
        if record:
            frames.append(env.render())

        # Выбор действия
        state_tensor = preprocess(state)
        logits = policy(state_tensor)
        # Преобразование логитов в вероятностное распределение и сэмплирование
        dist = torch.distributions.Categorical(logits=logits)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        log_probs.append(log_prob)

        # Шаг в среде
        state, reward, terminated, truncated, _ = env.step(action.item())
        done = terminated or truncated

        rewards.append(reward)
        total_reward += reward
        if done:
            break

    return total_reward, log_probs, rewards, frames

def update_policy(log_probs, rewards):
    # Проверка на пустые списки, если эпизод был слишком коротким.
    if not rewards:
        print("Предупреждение: Эпизод завершился слишком быстро, нет вознаграждений для обновления.")
        return

    # Вычисление дисконтированного суммарного вознаграждения (G_t)
    G = 0
    returns = []

    # Идем в обратном порядке
    for r in reversed(rewards):
        G = r + GAMMA * G
        returns.insert(0, G)

    returns = torch.tensor(returns)

    # Нормализация G_t
    if len(returns) > 1:
        returns = (returns - returns.mean()) / (returns.std() + 1e-9)

    # Вычисление функции потерь для Policy Gradient
    policy_loss = []
    for log_prob, Gt in zip(log_probs, returns):
        policy_loss.append(-log_prob * Gt)

    # Шаг градиентного спуска
    optimizer.zero_grad()
    loss = torch.stack(policy_loss).sum()

    loss.backward()
    optimizer.step()

In [8]:
#@title Обучение Policy Gradient

env_train = gym.make(GAME_NAME, full_action_space=FULL_ACTION_SPACE)
best_reward_pg = -1e9
best_policy_state = None

print("Запуск Policy Gradient для", GAME_NAME)

for ep in range(N_EPISODES):
    total_reward, log_probs, rewards, _ = run_episode_pg(env_train, policy)

    # Обновление политики
    update_policy(log_probs, rewards)

    if total_reward > best_reward_pg:
        best_reward_pg = total_reward
        # Сохраняем веса лучшей модели
        best_policy_state = policy.state_dict()
        print(f"[НОВЫЙ ЛУЧШИЙ] Эпизод {ep+1}/{N_EPISODES}: результат = {total_reward:.1f}")

    print(f"Эпизод {ep+1}/{N_EPISODES}:\tрезультат = {total_reward:.1f}\tлучший результат = {best_reward_pg:.1f}")

env_train.close()
print("Policy Gradient завершён.")

Запуск Policy Gradient для ALE/MsPacman-v5
[НОВЫЙ ЛУЧШИЙ] Эпизод 1/500: результат = 220.0
Эпизод 1/500:	результат = 220.0	лучший результат = 220.0
Эпизод 2/500:	результат = 190.0	лучший результат = 220.0
[НОВЫЙ ЛУЧШИЙ] Эпизод 3/500: результат = 240.0
Эпизод 3/500:	результат = 240.0	лучший результат = 240.0
Эпизод 4/500:	результат = 190.0	лучший результат = 240.0
[НОВЫЙ ЛУЧШИЙ] Эпизод 5/500: результат = 260.0
Эпизод 5/500:	результат = 260.0	лучший результат = 260.0
Эпизод 6/500:	результат = 200.0	лучший результат = 260.0
Эпизод 7/500:	результат = 160.0	лучший результат = 260.0
Эпизод 8/500:	результат = 230.0	лучший результат = 260.0
Эпизод 9/500:	результат = 250.0	лучший результат = 260.0
[НОВЫЙ ЛУЧШИЙ] Эпизод 10/500: результат = 270.0
Эпизод 10/500:	результат = 270.0	лучший результат = 270.0
Эпизод 11/500:	результат = 110.0	лучший результат = 270.0
Эпизод 12/500:	результат = 220.0	лучший результат = 270.0
Эпизод 13/500:	результат = 210.0	лучший результат = 270.0
Эпизод 14/500:	результа

In [21]:
#@title Видео результата Policy Gradient

print("Лучший результат Policy Gradient, достигнутый в тренировке:", best_reward_pg)

if best_policy_state:
    # Загружаем лучшую модель
    policy.load_state_dict(best_policy_state)

    # Запуск лучшего веса несколько раз для видео
    N_EVAL_RUNS = 5
    best_eval_reward_pg = -1e9
    best_eval_frames_pg = []

    env_render_pg = gym.make(GAME_NAME, full_action_space=FULL_ACTION_SPACE, render_mode="rgb_array")

    print(f"Проверка лучшей политики на {N_EVAL_RUNS} эпизодах для записи лучшего видео...")

    for i in range(N_EVAL_RUNS):
        # Используем обученную политику
        reward, _, _, frames = run_episode_pg(env_render_pg, policy, record=True)
        print(f"Оценочный прогон {i+1}: reward = {reward:.1f}")

        if reward > best_eval_reward_pg:
            best_eval_reward_pg = reward
            best_eval_frames_pg = frames

    env_render_pg.close()

    print(f"Максимальный результат для видео Policy Gradient: {best_eval_reward_pg:.1f}")

    show_video(best_eval_frames_pg, "best_policy_gradient.mp4")
else:
    print("Не удалось найти лучшую политику для видео.")

Лучший результат Policy Gradient, достигнутый в тренировке: 2680.0
Проверка лучшей политики на 5 эпизодах для записи лучшего видео...
Оценочный прогон 1: reward = 1140.0
Оценочный прогон 2: reward = 950.0
Оценочный прогон 3: reward = 1280.0
Оценочный прогон 4: reward = 760.0
Оценочный прогон 5: reward = 320.0
Максимальный результат для видео Policy Gradient: 1280.0
Moviepy - Building video best_policy_gradient.mp4.
Moviepy - Writing video best_policy_gradient.mp4



Moviepy - Done !
Moviepy - video ready best_policy_gradient.mp4


## Вывод по проделанной работе

Я применил алгоритм Policy Gradient для решения задачи управления в среде Gym Atari MsPacman-v5, используя PyTorch для построения нейронной сети политики. Я реализовал базовый алгоритм, который для обновления весов требует полного эпизода игры. В качестве оптимизации я применил нормализацию кумулятивных вознаграждений и использовал оптимизатор Adam с небольшой скоростью обучения.

Наблюдение за видеозаписями эпизодов и анализ финальных результатов показали, что производительность оказалась значительно ниже ожидаемой, а максимальное вознаграждение, достигнутое в тренировке, не было стабильно воспроизведено в оценочных прогонах. Главной проблемой стала высокая дисперсия метода: поскольку градиент основан на суммарном дисконтированном вознаграждении всей последующей траектории, даже небольшая случайность в игре MsPacman приводила к резким и нестабильным обновлениям политики. Отсутствие более совершенного механизм оценки также не позволяло эффективно снизить дисперсию, из-за чего агент не мог надежно отличить действительно хорошие действия от просто удачного эпизода. Таким образом, алгоритм Policy Gradient плохо подходит для этой задачи из-за высокой дисперсии и неэффективного использования данных.

При создании видео, мной было установлено, что лучший результат 2680.0 является выбросом, полученным за счет исключительно удачного стечения обстоятельств в высокодисперсном обучении. Политика, сохраненная с этим весом, не обладает обобщающей способностью, поскольку она оптимизирована под этот шумный, единичный эпизод, что объясняет, почему в оценочных прогонах результат падает до более стабильного уровня 1280.0.